# CW-Husky Glitch Testing

Runs some of the same glitch tests found in test_husky.py, but (much) longer. Besides the additional validation, this notebook can be helpful to diagnose any issues that may pop up.

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F3'
FORCE_LATEST_BITFILE = False

In [ ]:
if FORCE_LATEST_BITFILE:
    import chipwhisperer as cw

    try:
        if not scope.connectStatus:
            scope.con()
    except NameError:
        scope = cw.scope(name='Husky', bitstream="/home/jpnewae/git/cw_husky/fpga/vivado/cwhusky.runs/impl_no_ilas/cwhusky_top.bit")
        #scope = cw.scope(bitstream="/home/jpnewae/git/cw_husky/fpga/vivado/cwhusky.runs/impl_1/cwhusky_top.bit")

    try:
        if SS_VER == "SS_VER_2_1":
            target_type = cw.targets.SimpleSerial2
        elif SS_VER == "SS_VER_2_0":
            raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
        else:
            target_type = cw.targets.SimpleSerial
    except:
        SS_VER="SS_VER_1_1"
        target_type = cw.targets.SimpleSerial

    try:
        target = cw.target(scope, target_type)
    except:
        print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
        print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
        scope = cw.scope(bitstream="/home/jpnewae/git/cw_husky/fpga/vivado/cwhusky.runs/impl_no_ilas/cwhusky_top.bit")
        target = cw.target(scope, target_type)

    print("INFO: Found ChipWhisperer😍")

    import time
    time.sleep(0.05)
    scope.default_setup()
    def reset_target(scope):
        if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
            scope.io.pdic = 'low'
            time.sleep(0.1)
            scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
            time.sleep(0.1) #xmega needs more startup time
        elif "neorv32" in PLATFORM.lower():
            raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
        else:  
            scope.io.nrst = 'low'
            time.sleep(0.05)
            scope.io.nrst = 'high_z'
            time.sleep(0.05)
    reset_target(scope)

else:
    %run "../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
scope.fpga_buildtime

In [ ]:
from tqdm.notebook import tnrange
import random
import numpy as np

scope.adc.clip_errors_disabled = True
scope.adc.lo_gain_errors_disabled = True

def reset_setup():
    scope.trigger.module = 'basic'
    scope.trigger.triggers = 'tio4'
    scope.io.tio1 = "serial_rx"
    scope.io.tio2 = "serial_tx"
    scope.io.hs2 = "clkgen"
    scope.adc.timeout = 3
    scope.adc.offset = 0
    scope.glitch.enabled = False
    scope.LA.enabled = False
    scope.LA.clkgen_enabled = True
    scope.LA.capture_depth = 512
    scope.LA.downsample = 1
    scope.trace.enabled = False
    target.baud = 38400 * 10 / 7.37

def setup_glitch(offset, width, oversamp, LA=False):
    # set up glitch:
    scope.glitch.enabled = True
    scope.glitch.clk_src = 'pll'
    scope.clock.pll.update_fpga_vco(600e6)
    scope.glitch.repeat = 1
    scope.glitch.output = 'glitch_only'
    scope.glitch.trigger_src = 'ext_single'
    scope.glitch.offset = offset
    scope.glitch.width = width
    assert scope.glitch.mmcm_locked
    if LA:
        scope.LA.enabled = True
        scope.LA.oversampling_factor = oversamp
        scope.LA.capture_group = 'glitch'
        scope.LA.trigger_source = "glitch_source"
        scope.LA.capture_depth = 512
        assert scope.LA.locked
    else:
        scope.LA.enabled = True

def reset_target():
    scope.io.nrst = 0
    time.sleep(0.2)
    scope.io.nrst = 'high_z'
    time.sleep(0.2)

def find0to1trans(data):
    pattern = [0,1]
    return [i for i in range(0,len(data)) if list(data[i:i+len(pattern)])==pattern]

def test_missing_glitch_sweep_offset(clock, vco, span, width, num_glitches, reps, stepsize):
    # Checks for missing glitches (https://github.com/newaetech/chipwhisperer-husky-fpga/issues/4)
    setup_clock(clock)
    scope.clock.pll.update_fpga_vco(vco)
    scope.adc.samples = 16
    errors = []
    for offset in tnrange(scope.glitch.phase_shift_steps//2-span, scope.glitch.phase_shift_steps//2+span, stepsize):
        e = test_missing_glitch_single_offset(offset, width, num_glitches, reps)
        if e:
            error.append(e)
    assert errors == []

def setup_clock(clock):
    reset_setup()
    scope.clock.clkgen_freq = clock
    scope.clock.adc_mul = 1
    time.sleep(0.1)
    assert scope.clock.pll.pll_locked == True
    assert scope.clock.adc_freq == clock
    target.baud = 38400 * clock / 1e6 / 7.37
    reset_target()

def test_missing_glitch_single_offset(offset, width, num_glitches, reps):
    setup_glitch(offset, width, 1)
    scope.glitch.num_glitches = num_glitches
    errors = []
    for i in range(reps):
        ext_offsets = []
        for j in range(num_glitches):
            ext_offsets.append(random.randrange(2,5))
        scope.glitch.ext_offset = ext_offsets
        scope.glitch.repeat = [1]*num_glitches
        trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))
        assert trace is not None, 'capture failed (offset=%d, rep=%d)' % (offset, i)
        if scope.glitch.state != 'idle':
            errors.append(offset)
            print("ERROR: not in idle! state = %s, offset = %d, rep = %d" % (scope.glitch.state, offset, i))
            scope.glitch.state = None
    return errors


In [ ]:
# Test for missing glitches: run sweep around area of interest with lots of repetitions:
# This should take about 15 minutes:
test_missing_glitch_sweep_offset(clock=10e6, vco=1200e6, span=100, width=1000, num_glitches=10, reps=50, stepsize=1)

In [ ]:
# Test for missing glitches: run sweep around area of interest with lots of repetitions:
# This should take about 40 minutes:
test_missing_glitch_sweep_offset(clock=10e6, vco=600e6, span=3360, width=1000, num_glitches=10, reps=10, stepsize=1)

In [ ]:
# example to test a specific offset:
setup_clock(10e6)

In [ ]:
test_missing_glitch_single_offset(offset=1632, width=1000, num_glitches=10, reps=10)

In [ ]:
# To investigate behaviour using external logic analyzer:
scope.userio.mode = 'fpga_debug'
scope.fpga_reg_write(109, [5])